In [ ]:
# Import pandas
import pandas as pd

# Load your CSV file (replace 'your_file.csv' with your actual file path)
df_matrix = pd.read_csv("/home/labs/straussman/tatianas/Methylation_validation/Final_presence_absence_with_acessions_corrected_manually_DH_and_Tatiana.csv")

In [ ]:
df_matrix

In [ ]:
df_results = pd.read_excel("All_results_v2.xlsx")

In [ ]:
df_results

In [ ]:
#This code worked onto chatgpt:
import pandas as pd
import re

# === 1. Load input files ===
matrix_path = "Final_presence_absence_with_acessions_corrected_manually_DH_and_Tatiana.csv"
results_path = "All_results_v2.xlsx"

df_matrix = pd.read_csv(matrix_path)
df_results = pd.read_excel(results_path)

print("Matrix shape:", df_matrix.shape)
print("Results shape:", df_results.shape)


# === 2. Helpers to normalize IDs and HMM names ===
def normalize_seq_id(seq_id):
    """
    Normalize Sequence IDs to allow flexible matching.
    Handles UniProt-like IDs, metaeuk contig names, etc.
    """
    if pd.isna(seq_id) or seq_id == "no":
        return []
    
    # Split if multiple IDs are stored together
    parts = re.split(r"[|:/\s]+", str(seq_id).strip())
    
    tokens = []
    for p in parts:
        # remove trailing ".1", ".2", etc.
        p = re.sub(r"\.\d+$", "", p)
        if p:
            tokens.append(p)
    return tokens


def normalize_hmm_name(hmm):
    """Make HMM names consistent between files."""
    return re.sub(r"[^A-Za-z0-9]+", "_", str(hmm).strip().lower())


# === 3. Prepare df_results lookup ===
df_results["norm_seq_tokens"] = df_results["Sequence_ID"].apply(normalize_seq_id)
df_results["norm_hmm"] = df_results["HMM_used"].apply(normalize_hmm_name)


# === 4. Build dictionary for fast lookup ===
lookup = {}
for idx, row in df_results.iterrows():
    hmm = row["norm_hmm"]
    evalue = row["E-value"]
    for token in row["norm_seq_tokens"]:
        lookup.setdefault((token, hmm), []).append(evalue)


# === 5. Identify HMM / E-value column pairs in matrix ===
col_pairs = {}
for i, col in enumerate(df_matrix.columns):
    if col.startswith("E-value"):
        hmm_col = df_matrix.columns[i - 1]   # the HMM hit column is right before E-value
        evalue_col = col
        col_pairs[hmm_col] = evalue_col


# === 6. Fill the E-values in df_matrix ===
total_checked, matched, unmatched = 0, 0, 0
unmatched_records = []

for hmm_col, evalue_col in col_pairs.items():
    hmm_name = normalize_hmm_name(hmm_col)
    
    for idx, row in df_matrix.iterrows():
        seq_id = row[hmm_col]
        if pd.isna(seq_id) or seq_id == "no":
            continue
        
        total_checked += 1
        tokens = normalize_seq_id(seq_id)
        
        # Collect all possible matches
        candidate_evalues = []
        for token in tokens:
            candidate_evalues.extend(lookup.get((token, hmm_name), []))
        
        if candidate_evalues:
            # choose the best (lowest) e-value
            best_evalue = min(candidate_evalues, key=lambda x: float(x))
            df_matrix.at[idx, evalue_col] = best_evalue
            matched += 1
        else:
            unmatched += 1
            unmatched_records.append((seq_id, hmm_col))


# === 7. Save updated output ===
out_path = "updated_matrix_with_evalues_v4.csv"
unmatched_out = "unmatched_seq_hmm_v4.csv"

df_matrix.to_csv(out_path, index=False)
pd.DataFrame(unmatched_records, columns=["Sequence_ID", "HMM_col"]).to_csv(unmatched_out, index=False)

print("=== Done ===")
print("Checked:", total_checked)
print("Matched:", matched)
print("Unmatched:", unmatched)
print("Saved updated matrix to:", out_path)
print("Unmatched records saved to:", unmatched_out)


Matrix shape: (40, 43)
Results shape: (3897, 7)
=== Done ===
Checked: 493
Matched: 434
Unmatched: 59
Saved updated matrix to: updated_matrix_with_evalues_v3.csv
Unmatched records saved to: unmatched_seq_hmm_v3.csv
